In [25]:
import pandas as pd
import numpy as np
import networkx as nx
from pyvis.network import Network

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',100)
pd.set_option("display.max_colwidth", 135)

In [26]:
data = pd.read_csv("C:/Users/conno_52qz545/CS3220 AI/cs3220_2025f/data/game-of-thrones-battles.csv")
data.head()


,name,year,battle_number,attacker_king,defender_king,attacker_1,attacker_2,attacker_3,attacker_4,defender_1,defender_2,defender_3,defender_4,attacker_outcome,battle_type,major_death,major_capture,attacker_size,defender_size,attacker_commander,defender_commander,summer,location,region,note
0,Battle of the Golden Tooth,298,1,Joffrey/Tommen Baratheon,Robb Stark,Lannister,NaN,NaN,NaN,Tully,NaN,NaN,NaN,win,pitched battle,1.0,0.0,15000.0,4000.0,Jaime Lannister,"Clement Piper, Vance",1.0,Golden Tooth,The Westerlands,NaN
1,Battle at the Mummer's Ford,298,2,Joffrey/Tommen Baratheon,Robb Stark,Lannister,NaN,NaN,NaN,Baratheon,NaN,NaN,NaN,win,ambush,1.0,0.0,NaN,120.0,Gregor Clegane,Beric Dondarrion,1.0,Mummer's Ford,The Riverlands,NaN
2,Battle of Riverrun,298,3,Joffrey/Tommen Baratheon,Robb Stark,Lannister,NaN,NaN,NaN,Tully,NaN,NaN,NaN,win,pitched battle,0.0,1.0,15000.0,10000.0,"Jaime Lannister, Andros Brax","Edmure Tully, Tytos Blackwood",1.0,Riverrun,The Riverlands,NaN
3,Battle of the Green Fork,298,4,Robb Stark,Joffrey/Tommen Baratheon,Stark,NaN,NaN,NaN,Lannister,NaN,NaN,NaN,loss,pitched battle,1.0,1.0,18000.0,20000.0,"Roose Bolton, Wylis Manderly, Medger Cerwyn, Harrion Karstark, Halys Hornwood","Tywin Lannister, Gregor Clegane, Kevan Lannister, Addam Marbrand",1.0,Green Fork,The Riverlands,NaN
4,Battle of the Whispering Wood,298,5,Robb Stark,Joffrey/Tommen Baratheon,Stark,Tully,NaN,NaN,Lannister,NaN,NaN,NaN,win,ambush,1.0,1.0,1875.0,6000.0,"Robb Stark, Brynden Tully",Jaime Lannister,1.0,Whispering Wood,The Riverlands,NaN


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                38 non-null     object 
 1   year                38 non-null     int64  
 2   battle_number       38 non-null     int64  
 3   attacker_king       36 non-null     object 
 4   defender_king       35 non-null     object 
 5   attacker_1          38 non-null     object 
 6   attacker_2          10 non-null     object 
 7   attacker_3          3 non-null      object 
 8   attacker_4          2 non-null      object 
 9   defender_1          37 non-null     object 
 10  defender_2          2 non-null      object 
 11  defender_3          0 non-null      float64
 12  defender_4          0 non-null      float64
 13  attacker_outcome    37 non-null     object 
 14  battle_type         37 non-null     object 
 15  major_death         37 non-null     float64
 16  major_capt

In [28]:
battles_df=data.loc[:,['name','attacker_king','defender_king','attacker_size','defender_size']]
battles_df.head()

,name,attacker_king,defender_king,attacker_size,defender_size
0,Battle of the Golden Tooth,Joffrey/Tommen Baratheon,Robb Stark,15000.0,4000.0
1,Battle at the Mummer's Ford,Joffrey/Tommen Baratheon,Robb Stark,NaN,120.0
2,Battle of Riverrun,Joffrey/Tommen Baratheon,Robb Stark,15000.0,10000.0
3,Battle of the Green Fork,Robb Stark,Joffrey/Tommen Baratheon,18000.0,20000.0
4,Battle of the Whispering Wood,Robb Stark,Joffrey/Tommen Baratheon,1875.0,6000.0


In [29]:
battles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           38 non-null     object 
 1   attacker_king  36 non-null     object 
 2   defender_king  35 non-null     object 
 3   attacker_size  24 non-null     float64
 4   defender_size  19 non-null     float64
dtypes: float64(2), object(3)
memory usage: 1.6+ KB


In [30]:
battles_df_cleaned=battles_df.dropna()
battles_df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 0 to 37
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           16 non-null     object 
 1   attacker_king  16 non-null     object 
 2   defender_king  16 non-null     object 
 3   attacker_size  16 non-null     float64
 4   defender_size  16 non-null     float64
dtypes: float64(2), object(3)
memory usage: 768.0+ bytes


In [31]:
print(f"Attacking kings: {battles_df_cleaned.attacker_king.unique()}")

Attacking kings: ['Joffrey/Tommen Baratheon' 'Robb Stark' 'Stannis Baratheon']


In [32]:
print(f"Defending kings: {battles_df_cleaned.defender_king.unique()}")

Defending kings: ['Robb Stark' 'Joffrey/Tommen Baratheon' 'Balon/Euron Greyjoy'
 'Renly Baratheon' 'Mance Rayder']


In [33]:
net5kings = Network(heading="Task1. Building Interactive Network of battles of War of 5",
                    bgcolor = "#242020",
                    font_color = "white",
                    height = "1000px",
                    width = "100%",
                    directed = True, # we have directed graph
                    notebook = True,
                    cdn_resources = "remote"
)

In [34]:
type(data['attacker_king'].values)

numpy.ndarray

In [35]:
Kings_list=np.unique(np.concatenate((battles_df_cleaned['attacker_king'].values,battles_df_cleaned['defender_king'].values),0))
Kings_list

array(['Balon/Euron Greyjoy', 'Joffrey/Tommen Baratheon', 'Mance Rayder',
       'Renly Baratheon', 'Robb Stark', 'Stannis Baratheon'], dtype=object)

In [36]:
net5kings.add_nodes(Kings_list)
net5kings.nodes

[{'color': '#97c2fc',
  'id': 'Balon/Euron Greyjoy',
  'label': 'Balon/Euron Greyjoy',
  'shape': 'dot',
  'font': {'color': 'white'}},
 {'color': '#97c2fc',
  'id': 'Joffrey/Tommen Baratheon',
  'label': 'Joffrey/Tommen Baratheon',
  'shape': 'dot',
  'font': {'color': 'white'}},
 {'color': '#97c2fc',
  'id': 'Mance Rayder',
  'label': 'Mance Rayder',
  'shape': 'dot',
  'font': {'color': 'white'}},
 {'color': '#97c2fc',
  'id': 'Renly Baratheon',
  'label': 'Renly Baratheon',
  'shape': 'dot',
  'font': {'color': 'white'}},
 {'color': '#97c2fc',
  'id': 'Robb Stark',
  'label': 'Robb Stark',
  'shape': 'dot',
  'font': {'color': 'white'}},
 {'color': '#97c2fc',
  'id': 'Stannis Baratheon',
  'label': 'Stannis Baratheon',
  'shape': 'dot',
  'font': {'color': 'white'}}]

In [37]:
edges = battles_df_cleaned.loc[:,["attacker_king", "defender_king"]].values.tolist()
edges[:]

[['Joffrey/Tommen Baratheon', 'Robb Stark'],
 ['Joffrey/Tommen Baratheon', 'Robb Stark'],
 ['Robb Stark', 'Joffrey/Tommen Baratheon'],
 ['Robb Stark', 'Joffrey/Tommen Baratheon'],
 ['Robb Stark', 'Joffrey/Tommen Baratheon'],
 ['Robb Stark', 'Balon/Euron Greyjoy'],
 ['Joffrey/Tommen Baratheon', 'Robb Stark'],
 ['Robb Stark', 'Joffrey/Tommen Baratheon'],
 ['Stannis Baratheon', 'Renly Baratheon'],
 ['Joffrey/Tommen Baratheon', 'Robb Stark'],
 ['Robb Stark', 'Joffrey/Tommen Baratheon'],
 ['Stannis Baratheon', 'Joffrey/Tommen Baratheon'],
 ['Joffrey/Tommen Baratheon', 'Robb Stark'],
 ['Stannis Baratheon', 'Mance Rayder'],
 ['Stannis Baratheon', 'Balon/Euron Greyjoy'],
 ['Stannis Baratheon', 'Joffrey/Tommen Baratheon']]

In [38]:
edges_unique = []
for item in edges:
    if item not in edges_unique:
        edges_unique.append(item)
edges_unique

[['Joffrey/Tommen Baratheon', 'Robb Stark'],
 ['Robb Stark', 'Joffrey/Tommen Baratheon'],
 ['Robb Stark', 'Balon/Euron Greyjoy'],
 ['Stannis Baratheon', 'Renly Baratheon'],
 ['Stannis Baratheon', 'Joffrey/Tommen Baratheon'],
 ['Stannis Baratheon', 'Mance Rayder'],
 ['Stannis Baratheon', 'Balon/Euron Greyjoy']]

In [39]:
edge_weight = battles_df_cleaned.groupby(['attacker_king','defender_king']).size()
edge_weight

attacker_king             defender_king           
Joffrey/Tommen Baratheon  Robb Stark                  5
Robb Stark                Balon/Euron Greyjoy         1
                          Joffrey/Tommen Baratheon    5
Stannis Baratheon         Balon/Euron Greyjoy         1
                          Joffrey/Tommen Baratheon    2
                          Mance Rayder                1
                          Renly Baratheon             1
dtype: int64

In [40]:
Titles = battles_df_cleaned.groupby(["attacker_king","defender_king"])['name'].agg(', '.join)
Titles

attacker_king             defender_king           
Joffrey/Tommen Baratheon  Robb Stark                            Battle of the Golden Tooth, Battle of Riverrun, Sack of Winterfell, Battle of the Fords, The Red Wedding
Robb Stark                Balon/Euron Greyjoy                                                                                                 Battle of Torrhen's Square
                          Joffrey/Tommen Baratheon    Battle of the Green Fork, Battle of the Whispering Wood, Battle of the Camps, Battle of Oxcross, Sack of Harrenhal
Stannis Baratheon         Balon/Euron Greyjoy                                                                                                 Retaking of Deepwood Motte
                          Joffrey/Tommen Baratheon                                                                         Battle of the Blackwater, Siege of Winterfell
                          Mance Rayder                                                                  

In [41]:
edge_weight_df = edge_weight.reset_index(name = 'count')
Titles_df = Titles.reset_index()
Data_df = pd.merge(edge_weight_df, Titles_df, on=['attacker_king', 'defender_king'], how='inner')

def format_row(row):
    return f"Attacking King: {row['attacker_king']}  Defending King: {row['defender_king']} N of Battles: {row['count']} Battle Names: {row['name']}"

attacks = Data_df.apply(format_row, axis=1)
print(attacks)
print("edges_weight: ", edge_weight_df['count'].tolist())

0    Attacking King: Joffrey/Tommen Baratheon  Defending King: Robb Stark N of Battles: 5 Battle Names: Battle of the Golden Tooth, Batt...
1                  Attacking King: Robb Stark  Defending King: Balon/Euron Greyjoy N of Battles: 1 Battle Names: Battle of Torrhen's Square
2    Attacking King: Robb Stark  Defending King: Joffrey/Tommen Baratheon N of Battles: 5 Battle Names: Battle of the Green Fork, Battle...
3           Attacking King: Stannis Baratheon  Defending King: Balon/Euron Greyjoy N of Battles: 1 Battle Names: Retaking of Deepwood Motte
4    Attacking King: Stannis Baratheon  Defending King: Joffrey/Tommen Baratheon N of Battles: 2 Battle Names: Battle of the Blackwater,...
5                      Attacking King: Stannis Baratheon  Defending King: Mance Rayder N of Battles: 1 Battle Names: Battle of Castle Black
6                     Attacking King: Stannis Baratheon  Defending King: Renly Baratheon N of Battles: 1 Battle Names: Siege of Storm's End
dtype: object
edges_

In [ ]:
for index, row in Data_df.iterrows():
    net5kings.add_edge(row['attacker_king'], row['defender_king'], value=row['count'], title=row['name'])

def format_row(row):
    return f"The edge from '{row['attacker_king']}' to '{row['defender_king']}' with weight '{row['count']}' Title: '{row['name']}"

net5kings.edges


[{'Value': 5,
  'Title': 'Battle of the Golden Tooth, Battle of Riverrun, Sack of Winterfell, Battle of the Fords, The Red Wedding',
  'from': 'Joffrey/Tommen Baratheon',
  'to': 'Robb Stark',
  'arrows': 'to'},
 {'Value': 1,
  'Title': "Battle of Torrhen's Square",
  'from': 'Robb Stark',
  'to': 'Balon/Euron Greyjoy',
  'arrows': 'to'},
 {'Value': 5,
  'Title': 'Battle of the Green Fork, Battle of the Whispering Wood, Battle of the Camps, Battle of Oxcross, Sack of Harrenhal',
  'from': 'Robb Stark',
  'to': 'Joffrey/Tommen Baratheon',
  'arrows': 'to'},
 {'Value': 1,
  'Title': 'Retaking of Deepwood Motte',
  'from': 'Stannis Baratheon',
  'to': 'Balon/Euron Greyjoy',
  'arrows': 'to'},
 {'Value': 2,
  'Title': 'Battle of the Blackwater, Siege of Winterfell',
  'from': 'Stannis Baratheon',
  'to': 'Joffrey/Tommen Baratheon',
  'arrows': 'to'},
 {'Value': 1,
  'Title': 'Battle of Castle Black',
  'from': 'Stannis Baratheon',
  'to': 'Mance Rayder',
  'arrows': 'to'},
 {'Value': 1,
  

In [43]:
#Ignore this line of code

In [44]:
enemies_map = net5kings.get_adj_list()

for node in net5kings.nodes:
    node["value"] = len(enemies_map[node["id"]])+1

enemies_map

{'Balon/Euron Greyjoy': set(),
 'Joffrey/Tommen Baratheon': {'Robb Stark'},
 'Mance Rayder': set(),
 'Renly Baratheon': set(),
 'Robb Stark': {'Balon/Euron Greyjoy', 'Joffrey/Tommen Baratheon'},
 'Stannis Baratheon': {'Balon/Euron Greyjoy',
  'Joffrey/Tommen Baratheon',
  'Mance Rayder',
  'Renly Baratheon'}}

In [45]:
nodeColors={
    0:"blue",
    1:"green",
    2:"orange",
    3:"purple",
    4:"gold",
    5:"red"
}

In [46]:
for node in net5kings.nodes:
    node["color"] = nodeColors[node["value"]]

net5kings.nodes

[{'color': 'green',
  'id': 'Balon/Euron Greyjoy',
  'label': 'Balon/Euron Greyjoy',
  'shape': 'dot',
  'font': {'color': 'white'},
  'value': 1},
 {'color': 'orange',
  'id': 'Joffrey/Tommen Baratheon',
  'label': 'Joffrey/Tommen Baratheon',
  'shape': 'dot',
  'font': {'color': 'white'},
  'value': 2},
 {'color': 'green',
  'id': 'Mance Rayder',
  'label': 'Mance Rayder',
  'shape': 'dot',
  'font': {'color': 'white'},
  'value': 1},
 {'color': 'green',
  'id': 'Renly Baratheon',
  'label': 'Renly Baratheon',
  'shape': 'dot',
  'font': {'color': 'white'},
  'value': 1},
 {'color': 'purple',
  'id': 'Robb Stark',
  'label': 'Robb Stark',
  'shape': 'dot',
  'font': {'color': 'white'},
  'value': 3},
 {'color': 'red',
  'id': 'Stannis Baratheon',
  'label': 'Stannis Baratheon',
  'shape': 'dot',
  'font': {'color': 'white'},
  'value': 5}]

In [47]:
net5kings.show("lab1-task1-net5kings.html", notebook = False)

lab1-task1-net5kings.html
